In [38]:
#USAGE
#Upload the data file, set fileName variable
#Run all
#logs.csv is the required file
fileName = 'btcusdt_1h.csv'

In [39]:
!pip -q install arch

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from arch import arch_model
from itertools import product
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

import warnings
plt.style.use('seaborn-poster')
warnings.filterwarnings("ignore")

In [40]:
#from google.colab import drive
#drive.mount('/content/drive')


In [41]:
df = pd.read_csv("/content/"+fileName)
df["datetime"] = pd.to_datetime(df["datetime"])
df.set_index("datetime", inplace = True)
df.shape
df["return"] = df["close"].pct_change().shift(-1) * 100
df.fillna(0, inplace=True)

In [42]:
def arima_m(df, win):
    arima = np.zeros(df.shape[0])
    for i in range(0, df.shape[0], int(0.1*win)):
        try:
            arima_model = SARIMAX(np.log(df.close.iloc[i:i+win]), order=(1, 0, 1)).fit(disp=-1)
            arima[i+win: i+int(1.1*win)] = np.exp(arima_model.predict(start = win, end =int(1.1*win)-1))
        except:
            arima_model = SARIMAX(np.log(df.close.iloc[-int(1.1*win):-int(0.1*win)]), order=(1, 0, 1)).fit(disp=-1)
            arima[-int(0.1*win): ] = np.exp(arima_model.predict(start = win, end =int(1.1*win)-1))
    return arima

def garch_m(df, win):
    garch = np.zeros(df.shape[0])
    for i in range(0, win, 100):
      garch_model = arch_model(df['return'].iloc[i:int(i+100)], vol='EGARCH', p = 1, o = 0, q = 1, rescale = False).fit(options={'maxiter': 2000}, disp = 0)
      garch[i+100: i+110] = np.sqrt(np.array(garch_model.forecast(horizon = 10 , method='simulation', reindex=False).variance))
    for i in range(0, df.shape[0], int(0.1*win)):
        try:
            garch_model = arch_model(df['return'].iloc[i:int(i+win)], vol='EGARCH', p = 1, o = 0, q = 1, rescale = False).fit(options={'maxiter': 2000}, disp = 0)
            garch[i+win: i+int(1.1*win)] = np.sqrt(np.array(garch_model.forecast(horizon = int(0.1*win) , method='simulation', reindex=False).variance))
        except:
            garch_model = arch_model(df['return'].iloc[-int(1.1*win):-int(0.1*win)], vol='EGARCH', p = 1, o = 0, q = 1, rescale = False).fit(options={'maxiter': 2000}, disp = 0)
            garch[-int(0.1*win): ] = np.sqrt(np.array(garch_model.forecast(horizon = int(0.1*win) , method='simulation', reindex=False).variance))
    return garch

In [ ]:
df['garch'] = garch_m(df, 6000)

In [ ]:
win = 720
df['arima'] = arima_m(df, win)

In [ ]:
df['signals'] = np.zeros(df.shape[0])
i = win-1
gap = int(win/10) - 1
while i < (df.shape[0]-gap):
    d = df.arima.iloc[i+gap] - df.arima.iloc[i]
    if d > 200  and d < 5000 and df.garch.iloc[i] >0.5 and df.garch.iloc[i] < 4:
        df['signals'].iloc[i]  = 1
        df['signals'].iloc[i+gap] = -1
    elif d <-200 and d > -5000 and df.garch.iloc[i] >0.5 and df.garch.iloc[i] < 4 :
        df['signals'].iloc[i]  = -1
        df['signals'].iloc[i+gap] = +1
    i += gap+1

In [ ]:
df = df[['open', 'high', 'low', 'close', 'volume', 'signals']]
df.reset_index(inplace=True)
df['Index'] = df.index
df[['Index', 'datetime', 'open', 'high', 'low', 'close', 'volume', 'signals']].to_csv('/content/logs.csv',encoding='utf-8',index=False)